# Notebook 1: Structuring data without Neural Networks

**Authors:** Kenny Choo, Mark H. Fischer, Eliska Greplova for the conference "Summer School: ML in Quantum Physics and Chemistry" (24.08.-03.09.2021, Warsaw)

Adapted for the ML4Q-retreat 2022 by Alexander Gresch

In [ ]:
# Helper Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## Step 0. Loading and saving files with Google Colaboratory

Within these tutorials, we will need to upload some data (in particular, training data). E.g., today, we will upload Monte Carlo generated configurations of Ising spins on a 30x30 lattice. To upload data, you can pick one out of the three options (also, mix them freely):

- **Option A**. *Downloading data onto your Google drive and mounting the Google drive to this Colab notebook.*

"Mounting" means giving a temporary permission to a specific Colab notebook which enables browsing, loading, and saving files on your Google drive from this specific notebook. Once you exit the notebook, the perimission is gone, and you need to mount the Google drive again. Mounting is accompanied by an additional log-in to your Gmail account.

In [ ]:
# Option A.
# Mount your Google Drive:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Option A.
# Folder where you store the data (change if desired)
folder = '/content/drive/MyDrive/Colab Notebooks/retreat/'

# Access the data with simple commands:
ising_training_configs = np.load(folder + "/Ising/ising_training_configs_30x30.npy")

# You can also save them there:
x = np.array([0,1])
np.save(folder + "/first_save", x)

- **Option B.** *Clone the GitHub repository*

If you want to avoid working with Google drive, you can use this work-around. All needed data are on [this GitHub](https://github.com/GreschAl/ML4Q_retreat22_ML_with_python). You can clone this GitHub repository into your Colab environment in the same way as you would in your local machine, using git clone. Once the repository is cloned, refresh the file-explorer on the left to browse through its contents.

In [ ]:
# Option B.
!git clone https://github.com/GreschAl/ML4Q_retreat22_ML_with_python

In [ ]:
# Option B.
folder = "/content/ML4Q_retreat22_ML_with_python/exercises/Ising_data"
# Access the data with simple commands:
ising_training_configs = np.load(folder + "/ising_training_configs_30x30.npy")

Take note these are local and temporary files, and will disappear after closing the notebook. For saving and loading data from your local machine, look at Option C.

- **Option C.** *Downloading data onto your local machine and loading/saving data onto your local computer*

For saving and loading data from your local machine, you will need 'google.colab' library.

In [ ]:
# Option C.
from google.colab import files

# Saving data on your local machine:
x = np.array([0,1])
np.save('trial', x)
files.download('trial.npy')

In [ ]:
# Option C.
# Loading data from your local machine:
files.upload()  # it opens the file explorer where you can pick which files to upload 
                # (names stay the same as local ones)

Now we are ready to look at the data set!

# Example #1: Ising Spin Configuration Classification


The Ising model is given by the (classical) Hamiltonian:

\begin{align}
H(\boldsymbol{\sigma}) = -\sum_{<ij>} \sigma_{i}\sigma_{j},
\end{align}
where the spins $\sigma_{i} \in \lbrace -1, 1 \rbrace$ are binary variables living on the vertices of a square lattice and the sum is taken over nearest neighbours $<ij>$. We have set $J=1$.
  
At a given temperature $\beta = 1/T$, the probability of a configuration $\sigma$ is given by the Boltzmann distribution
  
\begin{align}
  P(\boldsymbol{\sigma}) = \frac{e^{-\beta H(\boldsymbol{\sigma})}}{Z},
 \end{align}
  
  where $Z$ is the partition function. This model exhibits a phase transition from the ferromagnetic phase at low tempertures to a paramagnetic phase at high temperatures. The transition temperature is $T_c \approx 2.2692$.
  
  **Task**
 
1.   Classify the ferromagnetic versus the paramagnetic phase of the Ising model
2.   Find the transition temperature
  
**Dataset**: Monte Carlo generated configurations on a 30x30 square lattice. The configuration are labelled by temperature.



## Step 1: Import data and analyze the data shape

The folder `Ising_data` contains Monte Carlo generated Ising configurations on the two-dimensional lattice. The data set is divided into training and test parts and corresponding label files containing the temperature, $T$, of each Monte Carlo sample.

In [ ]:
N = 30 # linear dimension of the lattice 

ising_training_configs = np.load(folder + "/ising_training_configs_{0}x{0}.npy".format(N))
ising_training_labels = np.load(folder + "/ising_training_labels_{0}x{0}.npy".format(N))
ising_test_configs = np.load(folder + "/ising_test_configs_{0}x{0}.npy".format(N))
ising_test_labels = np.load(folder + "/ising_test_labels_{0}x{0}.npy".format(N))

print('train_images.shape =', ising_training_configs.shape)
print('train_labels.shape =', ising_training_labels.shape)
print('test_images.shape =', ising_test_configs.shape)
print('test_labels.shape =', ising_test_labels.shape)

We see that we have a training set of size 1000 and a test set of size 1000.

Each image is a 30x30 array which takes values in {-1, 1}. The labels of these images are the temperatures and they are in [1, 3.5]. Let us plot some data to understand it further.

In [ ]:
def plot_Ising_configuration(spins, ax=None):
    '''
    this is just a helper function to plot the configuration of spins given by argument 'spins'
    '''
    if ax == None:
        print("problem")
        ax = plt.gca()
    N = np.shape(spins)[1]

    for i in range(N):
        ax.plot([i, i], [0, N-1], 'k', zorder=0, linewidth=0.5)
        ax.plot([0, N-1], [i,i], 'k', zorder=0, linewidth=0.5)

    cmap = plt.cm.Blues
    cmap2 = plt.cm.Oranges
    cmap(1)
    colors = [cmap2(140), cmap(220)] # note: orange is down, blue is up!
    colors = [cmap(220), cmap2(140)] # note: blue is down, orange is up!
    for i in range(N):
        for j in range(N):
            ax.add_patch(plt.Circle((i,j), radius=0.3, fc=colors[int((spins[i,j]+1)/2.)], zorder=2))

    ax.axes.set_axis_off()
    ax.set_ylim(-1, N+1)
    ax.set_xlim(-1, N+1)
    ax.set_aspect('equal')

In [ ]:
which_training_point1 = 50
which_training_point2 = 700

fig = plt.figure()
fig.set_size_inches(10, 30)
fig.subplots_adjust(wspace=0.01,hspace=0.08, left=0.01, right=0.99, bottom=0.05, top=0.99)

ax1 = fig.add_subplot(1,2,1)
plot_Ising_configuration(ising_training_configs[which_training_point1], ax1)
ax1.set_title("T={:1.2f}".format(ising_training_labels[which_training_point1]))
ax1.title.set_position((0.5,-0.1))

ax2 = fig.add_subplot(1,2,2)
plot_Ising_configuration(ising_training_configs[which_training_point2], ax2)
ax2.set_title("T={:1.2f}".format(ising_training_labels[which_training_point2]))
ax2.title.set_position((0.5,-0.1))

## Step 2: Use standard techniques to see a structure in the data
### Principal Component Analysis (PCA)

PCA is a dimensionality reduction method - it finds the features that vary the most accross the data set and then transforms the basis such that these features can be analyzed.


Here, we learn to do PCA on the above set of Ising configurations.

In [ ]:
# It is worth to implement it once by yourself, step by step.
# After this one time, use libraries like sklearn or tensorflow ;)
def pca(X=np.array([]), no_dims=50):
    """
          Runs PCA on the array X (shape NxD, where D is the size of a flattened sample)
          to reduce its dimensions to no_dims (usually start with no_dims=2, so you can visualize it easily)
    """
     
    ## We need to normalize our samples, e.g., make all pixel/site values across all samples have zero mean
    # You can start by extracting the dimensions of your data, i.e., number of samples and shape of each sample
    shape = X.shape

    ########################################################################
    ### TODO: ###
    ########################################################################
    # Calculate the mean value of each pixel/site of the configuration across all samples

    # Then, we substract the mean of each site over all samples, such that each site has a mean value of 0.

    ## Now we need to create the covariance matrix out of our data set

    # Diagonalise the covariance matrix    

    ## Finally, we take only the first 'no_dims' columns of the eigenvector matrix 
    ## and take the inner product with the input matrix X of all configurations.
    ########################################################################

    return inner_product

In [ ]:
########################################################################
### TODO: ###
########################################################################
# First we reshape the data such that we are working with matrices: 
# We flatten the configurations of 30x30 into 900 vector and create matrix, X, of all 1000 configurations: 
# This has a dimension 1000x900.
print("Before reshaping:", ising_training_configs.shape)
# X = ... 
print("After reshaping:", X.shape)
########################################################################

In [ ]:
# Now we can run our PCA on 1000 flattened samples
PCA_coord = pca(X, no_dims=2)

In [ ]:
# Here we plot the data points in the coordinates of the first two principal components
# with the color bar created using the temperature labels

fig = plt.figure()
fig.set_size_inches(7, 6)
fig.subplots_adjust(wspace=0.01,hspace=0.01, left=0.20, right=0.9, bottom=0.15, top=0.94)
ax1 = fig.add_subplot(1,1,1)

pca_plt = ax1.scatter(PCA_coord[:,0], PCA_coord[:,1], 10, ising_training_labels, 
            cmap=plt.cm.coolwarm)

cbar = plt.colorbar(pca_plt, )
cbar.set_label('Temperature, T')

ax1.set_xlabel('PCA component 1')
ax1.set_ylabel('PCA component 2')

# fig.savefig('pca_test.pdf')

### What we see in the plot?

Try and think for yourself before seeing the solution :)
E.g., how many clusters do you see and why?

The example above shows clear clusters. Thanks to the coloring the samples by the temperature that was used to generate them, we immediately see that the red, high temperature, cluster corresponds to the disordered phase and the light blue clusters on the boundaries correspond to two-types of ordered configurations: either all spins alighned up or all spins aligned down. It is possible to guess the value of the critical temperature from the color of the points placed between the clusters. Try to identify it and compare to the literature! 

# Example #2: Ising model with local constraints: Ising Gauge Theory (IGT)

In the previous example, we classified spin configurations of the simple Ising model. That was a relatively easy task given that we know that there's a global order parameter, i.e., the magnetization that distinguishes the two phases the model has.

In the following, we will look at spin configurations coming from a different model on which the simple PCA spectacularly fails. In this model, Ising spins live on the edges of a square lattice (see Figs. below). The Hamiltonian then favors even down and up spins around a square. If the number is odd, a pentalty is paid. The Hamiltonian is given by

\begin{align}
H(\boldsymbol{\sigma}) = -\sum_{p} \prod_{i \in p}\sigma_{i},
\end{align}
where we sum over the plaquettes $p$ of the square lattice.

This model does not have a finite temperature transition. We thus want to train a network to distinguish the (highly degenerate) ground states of this system from any excited state.

First, we load and analyze the shape of our data set again. As before, they are located in the folder `Ising` and labeled by a temperature.

In [ ]:
N = 16 # linear dimension of the lattice 

ilgt_training_configs = np.load(folder + "/ilgt_training_configs.npy".format(N))
ilgt_training_labels = np.load(folder + "/ilgt_training_labels.npy".format(N))
ilgt_test_configs = np.load(folder + "/ilgt_test_configs.npy".format(N))
ilgt_test_labels = np.load(folder + "/ilgt_test_labels.npy".format(N))

print('train_images.shape =', ilgt_training_configs.shape)
print('train_labels.shape =', ilgt_training_labels.shape)
print('test_images.shape =', ilgt_test_configs.shape)
print('test_labels.shape =', ilgt_test_labels.shape)

In [ ]:
def plot_igt_configuration(spins, name, dual=False, save=False):
    '''
    this is just a helper function to plot the configuration of spins given by 'spins'
    note that (i,j) denotes a vertex coordinate, such that the location of the plaquette
    center is at (i+0.5, j+0.5) and thus, the x spin is at (i+1, j+0.5) etc.

    Parameters
    ----------
    spins  :  int
        spin configuration, dimension is NxNx2
    dual   :  bool
        Plot the configuration in dual space or not. Default is False.
    '''
    N = np.shape(spins)[0]
    fig, ax = plt.subplots()
    fig.add_axes()
    ax = fig.axes[0]
    # spins = (2*spins-1)
    for i in range(N+1):
        ax.plot([i, i], [0,N], 'k')
        ax.plot([0,N], [i,i], 'k')

    if not dual:
        colors = ['#1329A4', '#F45A11'] # note: blue is down, gold is up!
        for i in range(N):
            fig.gca().add_patch(plt.Circle((0,i+0.5), radius=0.2, fc=colors[int((spins[-1,2*i]+1)/2.)]))
            fig.gca().add_patch(plt.Circle((i+0.5,0), radius=0.2, fc=colors[int((spins[i,-1]+1)/2.)]))
            for j in range(N):
                fig.gca().add_patch(plt.Circle((i+1,j+0.5), radius=0.2, fc=colors[int((spins[i,2*j+0]+1)/2.)]))
                fig.gca().add_patch(plt.Circle((i+0.5,j+1), radius=0.2, fc=colors[int((spins[i,2*j+1]+1)/2.)]))

    if dual:
        excitation = []
        for i in range(N):
            if spins[-1, 2*i+0]==1: ax.plot([-0.5, 0.5], [i+0.5, i+0.5], '#1329A4', lw=3)
            if spins[i, -1]==1: ax.plot([i+0.5, i+0.5], [-0.5, 0.5], '#1329A4', lw=3)
            for j in range(N):
                j_up = (N+j-1)%N
                i_left = (i+N-1)%N
                if spins[i,2*j+0]==1: ax.plot([i+0.5, i+1.5], [j+0.5, j+0.5], '#1329A4', lw=3)
                if spins[i,2*j+1]==1: ax.plot([i+0.5, i+0.5], [j+0.5, j+1.5], '#1329A4', lw=3)
                if spins[i,2*j+0]*spins[i_left, 2*j+0]*spins[i,2*j+1]*spins[i,2*j_up+1]==-1: excitation.append([i+0.5,j+0.5])
        if len(excitation)>0: plt.scatter(np.array(excitation)[:,0], np.array(excitation)[:,1], color='#F45A11', s=150, marker=(5,1))
    ax.set_ylim(-1,N+1)
    ax.set_xlim(-1,N+1)
    ax.set_aspect('equal')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.axis('off')
    plt.tight_layout()
    if save:
        plt.savefig("configuration_" + name +".pdf")

In [ ]:
# Notice indexing: (i,j) indexes plaquettes which have centers at (i+0.5, j+0.5).
# (0,0) is the bottom left corner of the lattice!
# Then you have two spin configurations.
# First spin is one on the right side of the plaquette (i+1,j+0.5),
# Second spin is one on the upper side of the plaquette (i+0.5,j+1)
print(ilgt_training_configs[0][0,0])

In [ ]:
# Let's plot the chosen sample. But first we need to reshape the data so the plotting procedure works
N = ilgt_training_configs[0].shape[0] # size of the lattice
# Pick one sample and reshape it to flatten the two sublattices (so you have N rows of 2*N spins)
sample = 0
spins = np.reshape(ilgt_training_configs[sample],[N,2*N])

#plt configuration
plot_igt_configuration(spins,'igt0')

#plot the dual mapping
plot_igt_configuration(spins,'igt0_dual',dual=True)
#ax1.set_title("T={:1.2f}".format(ising_training_labels[250]))
#ax1.title.set_position((0.5,-0.1))

Now we repeat PCA on the IGT traing data set:

In [ ]:
########################################################################
### TODO: ###
########################################################################
# firstly, flatten your data:
#X_ilgt = ...
########################################################################
PCA_coord_ILGT = pca(X_ilgt, no_dims=2)

#### Reshaping is not trivial, so here you have the solution if you're out of ideas

In [ ]:
X_ilgt = np.reshape(ilgt_training_configs, (ilgt_training_configs.shape[0], N*N*2))

### Plotting again!

In [ ]:
# Here we plot the data points in the coordinates of the first two principal components
# with the color bar created using the temperature labels

fig = plt.figure(dpi=200)
fig.set_size_inches(3.40457, 3)
fig.subplots_adjust(wspace=0.01,hspace=0.01, left=0.20, right=0.9, bottom=0.15, top=0.94)
ax1 = fig.add_subplot(1,1,1)

pca_plt = ax1.scatter(PCA_coord_ILGT[:,0], PCA_coord_ILGT[:,1], 10, ilgt_training_labels, 
            cmap=plt.cm.coolwarm)

cbar = plt.colorbar(pca_plt, )
cbar.set_label('Temperature, T')

ax1.set_xlabel('PCA component 1')
ax1.set_ylabel('PCA component 2')

### What do you see and what do you think about it?

Check the answer here:

Wow - by introducing a simple constraint to the Hamiltonian, we broke the PCA. It turns out that going just a little bit outside of the simple models makes straighforward clustering difficult to work. 


## Extra credit assignment
You can go ahead and try more sophisticated method to test the difference between standard Ising model and IGT. One such example is t-Stochastic Neighborhood Embedding (t-SNE). Spoiler alert: it won't help you...

# t-SNE for Ising and Ising with constraints models (SOLUTION)

In [ ]:
# import t-SNE method from skikit learn
from sklearn.manifold import TSNE as tsne

In [ ]:
# the input should be the same as for PCA
## Ising model
N = ising_training_configs.shape[1] #size of conf
########################################################################
### TODO: ###
########################################################################
# flatten the Ising data once again (can copy code from above, of course)
# X = ...

# repeat for the other data set as well
## Ising Gauge Theory model
N_igt = ilgt_training_configs[0].shape[0]

# X_igt = ...
# Ensure that each pixel has zero mean
########################################################################

In [ ]:
use_Ising_data = True # easy switch for changing data sets

data = X if use_Ising_data else X_igt
########################################################################
### TODO: ###
########################################################################
# have a look at the docs of the function tsne() and how to use it
# in order to transform <data>. Choose 2 output dimensions once again.
# tsne_fits = ...
########################################################################

In [ ]:
fig = plt.figure(dpi=200)
fig.set_size_inches(3.40457, 3)
fig.subplots_adjust(wspace=0.01,hspace=0.01, left=0.20, right=0.9, bottom=0.15, top=0.94)
ax1 = fig.add_subplot(1,1,1)

labels = ising_training_labels if use_Ising_data else ilgt_training_labels
tsne_plt = ax1.scatter(tsne_fits[:,0], tsne_fits[:,1], 10, labels, 
            cmap=plt.cm.coolwarm)


cbar = plt.colorbar(tsne_plt, )
cbar.set_label('Temperature, T')

ax1.set_xlabel('tSNE component 1')
ax1.set_ylabel('tSNE component 2')